In [1]:
import os
import glob
import tifffile
from cellpose import utils, io
from skimage.io import imsave, imread



In [2]:
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
import tifffile
import os

In [6]:
project_dir = ("/scratch/zqn7td/20250626_HetMel_ImageProcessing/")


In [6]:
case_numbers = [
    "Case1-31",]

In [7]:
case_numbers = [
    "Case1-31",
    "Case2-8",
    "Case3-9",
    "Case4-5_R1",
    "Case4-5_R2",
    "Case7-10",
    "Case8-9_R1",
    "Case8-9_R2",
    "Case9-7_R2",
    "Case10-3_R1",
    "Case10-3_R2",
    "Case11-13"
]


In [8]:

for case in case_numbers:
    case_dir = os.path.join(project_dir, case)
    mask_file = glob.glob(os.path.join(case_dir, "**", "*segmask*.tif"), recursive=True)
    mask_path = mask_file[0]
    print(f"processing {mask_path}")
    
    masks = io.imread(mask_path)
    dilate_masks_with_ring(mask_path,pixels=4,prefix=f"{case}_segmask")
    

processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case1-31/Case1-31_bgsub2_ome_rotate-crop.ome_dia10.0_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case2-8/Case2-8_new_bgsub_final.ome_dia10_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case3-9/Case3-9_new_bgsub_new_rotate-crop_final.ome_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case4-5_R1/Case4-5_R1_bgsub_rotate-crop.ome_dia12.0_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case4-5_R2/Case4-5_R2_bgsub_rotate-crop.ome_dia12.0_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case7-10/Case7-10_new_bgsub_new_rotate-crop_final.ome_dia10_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case8-9_R1/Case8-9_R1_new_bgsub_rotate-crop_final.ome_dia10_cp_segmasks.tif
processing /scratch/zqn7td/20250626_HetMel_ImageProcessing/Case8-9_R2/Case8-9_R2_new_bgsub_rotate-crop_f

find image and mask paths

In [15]:
mask_path

'/scratch/zqn7td/20250626_HetMel_ImageProcessing/Case1-31/Case1-31_bgsub2_ome_rotate-crop.ome_dia10.0_cp_segmasks.tif'

In [17]:
masks = io.imread(mask_path)

In [21]:
print(masks)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
cyto_mask = utils.dilate_masks(masks, 3)

KeyboardInterrupt: 

In [ ]:
# Recursively find all segmask tif files in all subfolders
mask_paths = glob.glob(os.path.join(project_dir, "**", "*segmask*.tif"), recursive=True)

In [5]:


def dilate_masks_with_ring(input_path, pixels=5, save_path=None, prefix="mask"):
    """
    Expand nucleus masks outward by n_iter pixels to create cytoplasmic rings,
    with no overlaps between cells. Input and outputs are TIFFs.

    Args:
        input_path (str): Path to input segmentation mask (TIFF).
                          Must be a labeled mask (0 = background, 1..N = objects).
        n_iter (int): Ring thickness in pixels.
        save_path (str, optional): Directory to save output TIFFs. 
                                   Defaults to the same directory as input_path.
        prefix (str, optional): Prefix for saved filenames.

    Returns:
        expanded (ndarray): Labeled masks including nucleus + cytoplasm.
        rings (ndarray): Labeled cytoplasmic rings only (no nuclei).
    """
    # Load mask as ndarray
    masks = tifffile.imread(input_path).astype(np.int32)

    # Distance to the nearest nucleus
    dist = distance_transform_edt(masks == 0)

    # Watershed ensures unique expansion with no overlaps
    expanded = watershed(-dist, markers=masks, mask=dist <= pixels)

    # Cytoplasmic ring = expanded region minus nucleus
    rings = np.where((expanded != 0) & (masks == 0), expanded, 0)

    # If no save_path is provided, default to input file’s directory
    if save_path is None:
        save_path = os.path.dirname(input_path)
    os.makedirs(save_path, exist_ok=True)

    # Save outputs as TIFF
    tifffile.imwrite(f"{save_path}/{prefix}_nuc_cyto.tif", expanded.astype(np.uint16))
    tifffile.imwrite(f"{save_path}/{prefix}_cytorings.tif", rings.astype(np.uint16))

    return expanded, rings


In [24]:
mask_path

'/scratch/zqn7td/20250626_HetMel_ImageProcessing/Case1-31/Case1-31_bgsub2_ome_rotate-crop.ome_dia10.0_cp_segmasks.tif'

In [27]:
dilate_masks_with_ring(mask_path,n_iter=3)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32))